# Vergleich und Visualisierung der Embeddings

Du hast jetzt unterschiedliche Embeddings kennengelernt. Jetzt möchtest du deren Ergebnis miteinander vergleichen. Dazu gibt es verschiedene Visualisierungen, du wirst dir dazu zwei ganz unterschiedliche grafische Darstellungen anschauen.

## Embeddings einladen

Damit du die Embeddings nicht nochmal berechnen musst, lädst du die vorher gespeicherten Wortvektoren einfach ein:

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.w2v || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.w2v.gz && gunzip heise-articles-2020.w2v.gz")
    os.system("test -f heise-articles-2020.ft || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.ft.gz && gunzip heise-articles-2020.ft.gz")
    os.system("test -f heise-articles-2020.glove.txt || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.glove.txt.gz && gunzip heise-articles-2020.glove.txt.gz")
    w2v_file = "heise-articles-2020.w2v"
    ft_file = "heise-articles-2020.ft"
    glove_file = "heise-articles-2020.glove.txt"
else:
    w2v_file = "../99_Common/heise-articles-2020.w2v"
    ft_file = "../99_Common/heise-articles-2020.ft"
    glove_file = "../99_Common/heise-articles-2020.glove.txt"

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format(w2v_file)

In [ ]:
ft = KeyedVectors.load_word2vec_format(ft_file)

In [ ]:
glove = KeyedVectors.load_word2vec_format(glove_file, no_header=True)

## 2D-Plots

Zur Visualisierung kannst du dir die Word Embeddings leider nicht alle ausgeben lassen, denn das sind ziemlich viele:

In [ ]:
len(w2v.key_to_index)

Stattdessen kannst du dich auf die Keywords konzentrieren, am besten auch nur auf die Top-Keywords. Die Berechnung der Häufigkeiten kennst du schon:

In [ ]:
if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT keywords FROM articles WHERE datePublished<'2021-01-01'", sql)

In [ ]:
from collections import Counter
keywords = Counter([keyword for keywords in df["keywords"] for keyword in str(keywords).split(", ")])

Dabei solltest du darauf achten, dass keine Leerzeichen enthalten sind, weil die bei den Embeddings als unterschiedliche Wörter interpretiert wurden:

In [ ]:
top_keywords = [k[0] for k in keywords.most_common(100) if not " " in k[0]][0:50]

Jetzt hast du die Anzahl der Embeddings reduziert - aber leider haben sie noch immer 300 Dimensionen. Eine sinnvolle Darstellung kannst du damit nicht erreichen. Allerdings kannst du die [Anzahl der Dimensionen reduzieren](https://en.wikipedia.org/wiki/Dimensionality_reduction). Dafür gibt es verschiedene Verfahren, hier benutzt du [T-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).

Zunächst berechnest du eine Matrix mit (möglichst) allen Embeddings der Keywords. Dabei können einzelne fehlen, die durch Exception-Handling dann leer sind. Anschließend führst du mit `TSNE` eine Dimensionsreduktion durch und stellst das Ergebnis in zwei Dimensionen als Scatter-Plot dar. Schließlich muss der Scatter-Plot noch beschriftet werden.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_wv(model, words):
    wr = []
    wk = []
    for w in words:
        # manche Wörter könnten fehlen
        try:
            wr.append(model[w.lower()])
            wk.append(w)
        except:
            pass

    tsne = TSNE(n_components=2, random_state=42).fit_transform(wr)
    tsne_df = pd.DataFrame(tsne, columns = ["x", "y"])
    
    ax = tsne_df.plot.scatter(x='x', y='y', figsize=(16, 9))

    for i, txt in enumerate(wk):
        ax.annotate(txt, (tsne_df.x[i], tsne_df.y[i]))

Betrachte nun, wie sich die `word2vec`-Embeddings verhalten:

In [ ]:
plot_wv(w2v, top_keywords)

Und zum Vergleich dazu die von `fastText`:

In [ ]:
plot_wv(ft, top_keywords)

`glove` ordnet etwas anders und packt die Länder enger zusammen:

In [ ]:
plot_wv(glove, top_keywords)

Über diese zweidimensionalen Darstellungen kannst du dir einen Überblick verschaffen, wie die Wörter in den einzelnen Embeddings zueinander stehen. Beachte dabei, dass als Ähnlichkeitsmaß normalerweise der Winkel zwischen Wörtern verwendet wird, der hier durch die Dimensionsreduktion sicher sehr verzerrt wurde!

## Netzwerk

Ähnlichkeiten kannst du auch ganz anders darstellen - nämlich als Netzwerk, bei dem du von einem zentralen Wort ausgehst, zzu diesem die ähnlichsten Wörter suchst usw. Dadurch ergibt sich ein *Graph*.

Python hat mit `networkx` eine ziemlich flexible Bibliothek zur Verarbeitung und auch zur Darstellung von Graphen. Dazu schreibst du dir zunächst eine Funktion, die den Graph mit den Wörtern als Knoten aufbauen und die Kanten zwischen ähnlichen Wörtern aufspannt:

In [ ]:
!pip install networkx

In [ ]:
import networkx as nx

def graph(model, w0):
    G=nx.Graph()
    G.add_node(w0)
    for (w1, s1) in model.most_similar(w0):
        G.add_node(w1)
        G.add_edge(w0, w1, weight=s1)
        for (w2, s2) in model.most_similar(w1):
            G.add_node(w2)
            G.add_edge(w1, w2, weight=s2)
            
    return G

Betrachtet nun die Ähnlichkeiten von `apple` om Graph mit `word2vec`:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(w2v, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

Im  Vergleich mit `fastText` fällt auf, dass hier viel mehr Ähnlichkeiten gefunden werden, die auf ähnliche Schreibweisen zurückzuführen sind:

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(ft, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

Bei `glove` sieht schon die Form ganz anders aus. Wie du siehst, sind deutlich mehr allgemeine Wörter enthalten, die wenig spezifische Bedeutung transportieren.

In [ ]:
plt.figure(3,figsize=(12,12)) 
G = graph(glove, "apple")
nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')

## Word Embeddings verhalten sich unterschiedlich

Wie du siehst verhalten sich die Embeddings sehr verschieden. `word2vec` fängt eher konzeptionelle Ähnlichkeiten ein, während `fastText` sich mehr auf Syntax konzentriert. `glove` produziert teilweise sehr gute Resultate, teilweise aber auch nur ganz allgemeine Wörter.

Welches Embedding für dich am besten geeignet ist, musst du anhand deiner Anforderungen selbst herausfinden. Die oben gezeigten Visualisierungen können dich dabei unterstützen.